In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-06 19:01:29.152912


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-03-06T19:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(weeks=52)

In [7]:
fetch_data_to

Timestamp('2025-03-06 19:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2024-03-07 19:00:00+0000', tz='UTC')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)
    # - timedelta(weeks=52)

    print(historical_from_date)
    print(historical_to_date)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year)
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date.to_datetime64()]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date.to_datetime64()]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)
    print(rides['pickup_datetime'])

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

2023-03-09 19:00:00+00:00
2024-03-07 19:00:00+00:00
File already exists for 2023-01.
Loading data for 2023-01...
Total records: 3,066,766
Valid records: 2,993,140
Records dropped: 73,626 (2.40%)
Successfully processed data for 2023-01.
File already exists for 2023-02.
Loading data for 2023-02...
Total records: 2,913,955
Valid records: 2,845,058
Records dropped: 68,897 (2.36%)
Successfully processed data for 2023-02.
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 3,403,766
Valid records: 3,331,705
Records dropped: 72,061 (2.12%)
Successfully processed data for 2023-03.
File already exists for 2023-04.
Loading data for 2023-04...
Total records: 3,288,250
Valid records: 3,214,922
Records dropped: 73,328 (2.23%)
Successfully processed data for 2023-04.
File already exists for 2023-05.
Loading data for 2023-05...
Total records: 3,513,649
Valid records: 3,435,875
Records dropped: 77,774 (2.21%)
Successfully processed data for 2023-05.
File already exists for 2023

In [11]:
rides.head(5)

,pickup_datetime,pickup_location_id
8776023,2024-03-27 10:17:09,2
9056155,2024-03-29 19:59:30,2
9206932,2024-03-30 12:11:27,2
15501963,2024-05-27 20:03:00,2
16214443,2024-06-02 15:51:11,2


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data
len(ts_data)

1854580

In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854580 entries, 0 to 1854579
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 21.2 MB


In [15]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time series data at hourly frequency v4",
    primary_key=['pickup_location_id','pickup_hour'],
    event_time = ['pickup_hour']
)

2025-03-06 14:03:03,238 INFO: Initializing external client
2025-03-06 14:03:03,239 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 14:03:04,181 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648


In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214648/fs/1202279/fg/1403640


Uploading Dataframe: 100.00% |████████████████████████████████████████████████| Rows 1854580/1854580 | Elapsed Time: 01:22 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_v4_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214648/jobs/named/time_series_hourly_feature_group_v4_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_v4_1_offline_fg_materialization', 'SPARK'),
 None)